In [101]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import natsort
import re
#We gebruiken de from ... import voor commando's die we veel gebruiken om repititie te verminderen 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from zipfile import ZipFile
import tensorflow as tf

UPLOAD EERST DE DATA ZIP!

In [102]:
standaarddirectory = os.getcwd()                  #standaarddirectory van google colab opslaan zodat we altijd terug kunnen keren naar deze
file_name = "delay_dataset.zip"                   
if not os.path.exists("stud_train"):              #Als de data nog niet uitgepakt is en deze folder dus nog niet bestaat, gaan we dit doen
  with ZipFile(file_name,'r') as zip:
    zip.extractall()
    print("extractie data klaar")

if not os.path.exists("stud_train_combined_ASDR"):    #Een folder waarin we de gecombineerde ASDR signalen van 1024 samples zullen opslaan
  os.makedirs("stud_train_combined_ASDR")
if not os.path.exists("stud_train_combined_GAUSS"):   #Een folder waarin we de gecombineerde GAUSS signalen van 1024 samples zullen opslaan
  os.makedirs("stud_train_combined_GAUSS")
if not os.path.exists("stud_train_combined_SBOX"):    #Een folder waarin we de gecombineerde SBOX signalen van 1024 samples zullen opslaan
  os.makedirs("stud_train_combined_SBOX")

In [103]:
# variable declaration
train_set = 'stud_train'
test_set = 'stud_test'
delaypiekASDR = []    #array met actuele delays tussen de pieken (validatiecijfers voor training)
delaypiekGAUSS = []
delaypiekSBOX = []

In [104]:
#Geeft een array met strings weer waarin de namen van de files in die map staan
def laadarraymetfilenamenuitmap(map):
    filenames = os.listdir(map)  # array met alle filenames van in folder
    filenames.sort()
    return filenames

In [105]:
#plot het signaal van een numpy array aan de hand van een megegeven signaalbestandsnaam
def plotsignaal(signaal, evencheck, nummer, x = np.arrange(0,512)): #signaal is naam bestand tussen aanhalingstekens
    if evencheck % 2 == 0:
        kleur = "red"         #paar van signalen respectievelijk rood en blauw weergeven zodat het verschil duidelijk is
    else:
        kleur = "blue"
    plot = plt.figure(nummer)
    plt.xlabel("X axis")
    plt.ylabel("Y axis")
    plt.plot(x,np.load(signaal), color=kleur)

In [106]:
#Geeft de waarde van de positie van die piek terug aan de hand van de bestandsnaam
def geefpositiepiek(signaal):
  truncatedsignaal = (signaal.split("P_",1)[1]) #alles voor de xxx wegdoen dus BV ASDR_P_ wegdoen
  piek = ''.join(x for x in truncatedsignaal[0:3] if x.isdigit()) 
  return piek

In [107]:
#voegt de array van twee signalen samen om 1 vector van 1024 samples te bekomen
def voegsamen(signaal1, signaal2):
  return np.concatenate([np.load(signaal1), np.load(signaal2)])

In [108]:
#wordt gebruikt om de lijst met de files te sorten op numerische orde ipv alfabetische orde om sortering van 0 1 10 11 ... 19 2 20 21 ... 29 3 30 31 etc te voorkomen maar 0 1 2 3 4 ... 99
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [109]:
def combinedata(folder, dir, delayarray, aantalgrafiekenplotten = 0):
  i = 0
  nummer = 1
  evencheck = 0
  x = np.arange(0, 1024)
  k = 0
  for data in folder:
    evencheck += 1
    if k != 0 and evencheck % 2 == 0:
      os.chdir("stud_train")
      masterfile = voegsamen(previous, data)
      os.chdir(standaarddirectory)

      os.chdir(dir)
      savename = f"{nummer}.npy"
      np.save(savename,masterfile)
      os.chdir(standaarddirectory)
      nummer += 1
      delayarray.append(512-int(geefpositiepiek(previous))+int(geefpositiepiek(data)))

      #plotten voor eerste paar grafieken, mooie visualisatie als controle
      if evencheck < ((aantalgrafiekenplotten*2)+1):  #evencheck > aantal grafieken*2, max 41 vanwege memory limiet -> 20 grafieken, zet op 1 indien je geen grafieken wil zien
        plot= plt.figure(nummer)
        plt.plot(x,masterfile, color="red")

    k += 1
    previous = data

In [110]:
#main
#filenames inlezen
filenames = laadarraymetfilenamenuitmap(train_set)
filenamesASDR = filenames[0:200]
filenamesGAUSS = filenames[200:400]
filenamesSBOX = filenames[400:600]

#sorteren
filenamesASDR.sort(key=natural_keys)
filenamesGAUSS.sort(key=natural_keys)
filenamesSBOX.sort(key=natural_keys)

In [111]:
combinedata(filenamesASDR, "stud_train_combined_ASDR", delaypiekASDR)
combinedata(filenamesGAUSS, "stud_train_combined_GAUSS", delaypiekGAUSS)
combinedata(filenamesSBOX, "stud_train_combined_SBOX", delaypiekSBOX)

#ASDRCOMBINEDFILES = os.listdir("stud_train_combined_ASDR")
#natsort.natsorted(ASDRCOMBINEDFILES)

In [112]:
#TENSOR Object maken voor x waarden NN training
os.chdir(standaarddirectory)
os.chdir("stud_train_combined_ASDR")

samples = [np.load(f"{x}.npy") for x in range(1, 101)]
samples = tf.convert_to_tensor(samples)

os.chdir(standaarddirectory)

In [113]:
#TENSOR object maken voor y waarden NN training
dfdelay = pd.DataFrame(data = delaypiekASDR)
dfdelay = tf.convert_to_tensor(dfdelay)

In [114]:
#Model aanmaken
model = tf.keras.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(1024,)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1))

In [115]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 512)               524800    
                                                                 
 dense_26 (Dense)            (None, 256)               131328    
                                                                 
 dense_27 (Dense)            (None, 64)                16448     
                                                                 
 dense_28 (Dense)            (None, 8)                 520       
                                                                 
 dense_29 (Dense)            (None, 1)                 9         
                                                                 
Total params: 673,105
Trainable params: 673,105
Non-trainable params: 0
_________________________________________________________________


In [116]:
#Optimizer, loss function en metrics toevoegen aan model en compileren
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

In [117]:
#Model Trainen
history = model.fit(samples,
                    dfdelay,
                    epochs=500)

Epoch 1/500
4/4 [==============================] - 1s 9ms/step - loss: 576.0588 - mae: 576.0588
Epoch 2/500
4/4 [==============================] - 0s 8ms/step - loss: 573.7961 - mae: 573.7961
Epoch 3/500
4/4 [==============================] - 0s 9ms/step - loss: 567.1016 - mae: 567.1016
Epoch 4/500
4/4 [==============================] - 0s 9ms/step - loss: 551.8911 - mae: 551.8911
Epoch 5/500
4/4 [==============================] - 0s 12ms/step - loss: 520.5994 - mae: 520.5994
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 461.3031 - mae: 461.3031
Epoch 7/500
4/4 [==============================] - 0s 9ms/step - loss: 357.9292 - mae: 357.9292
Epoch 8/500
4/4 [==============================] - 0s 8ms/step - loss: 190.8161 - mae: 190.8161
Epoch 9/500
4/4 [==============================] - 0s 8ms/step - loss: 95.4166 - mae: 95.4166
Epoch 10/500
4/4 [==============================] - 0s 9ms/step - loss: 143.2004 - mae: 143.2004
Epoch 11/500
4/4 [=====================

In [118]:
predictions = model.predict(samples)
print(predictions)
print(delaypiekASDR)

4/4 [==============================] - 0s 4ms/step
[[577.8982 ]
 [562.75586]
 [606.03687]
 [574.1122 ]
 [574.35486]
 [635.5511 ]
 [620.0692 ]
 [579.2833 ]
 [561.2186 ]
 [596.5487 ]
 [617.78107]
 [585.97174]
 [557.5502 ]
 [563.9278 ]
 [617.776  ]
 [608.159  ]
 [592.02246]
 [581.8537 ]
 [589.33545]
 [578.4209 ]
 [554.1737 ]
 [582.26746]
 [581.0415 ]
 [578.084  ]
 [621.4565 ]
 [562.48584]
 [602.2814 ]
 [532.5249 ]
 [558.6592 ]
 [551.7224 ]
 [569.08026]
 [562.6827 ]
 [571.24744]
 [591.40845]
 [594.6146 ]
 [619.14276]
 [578.4326 ]
 [571.971  ]
 [603.4746 ]
 [595.87866]
 [577.64197]
 [575.6583 ]
 [541.54486]
 [597.5632 ]
 [626.4291 ]
 [548.6182 ]
 [583.3997 ]
 [581.6928 ]
 [606.88873]
 [569.7134 ]
 [600.53656]
 [566.82684]
 [560.8045 ]
 [561.1928 ]
 [604.7182 ]
 [622.22394]
 [595.57513]
 [610.62646]
 [622.8558 ]
 [556.28326]
 [565.5609 ]
 [551.14813]
 [624.4261 ]
 [546.3411 ]
 [600.4333 ]
 [603.9248 ]
 [565.56244]
 [572.90106]
 [619.49695]
 [607.96173]
 [540.2073 ]
 [585.6177 ]
 [599.4748 ]


In [119]:
#reset